# Decision analysis with Julia `Agents.jl` using Petitti, 2000, 2e. - Chapter 2 
Tomás Aragón, Updated 2026-01-21

Here is the decision tree from Petitti, 2000, 2e, Chapter 2, Figure 2-6 (FIGURE 1). 

In [1]:
import Pkg
# 1. Activate a new folder for this project
Pkg.activate("DecisionAnalysisProject") 

# 2. Force install the latest version
Pkg.add("Agents") 

# 3. Check the version (should be v6+)
Pkg.status("Agents")

  Activating project at `~/Library/CloudStorage/Dropbox/tja/ACE/Efforts/Personal/2_projects/di4h/gh_di4h/notebooks/Petitti_2000_2e_ch02/DecisionAnalysisProject`
   Resolving package versions...
     Project No packages added to or removed from `~/Library/CloudStorage/Dropbox/tja/ACE/Efforts/Personal/2_projects/di4h/gh_di4h/notebooks/Petitti_2000_2e_ch02/DecisionAnalysisProject/Project.toml`
    Manifest No packages added to or removed from `~/Library/CloudStorage/Dropbox/tja/ACE/Efforts/Personal/2_projects/di4h/gh_di4h/notebooks/Petitti_2000_2e_ch02/DecisionAnalysisProject/Manifest.toml`


Status `~/Library/CloudStorage/Dropbox/tja/ACE/Efforts/Personal/2_projects/di4h/gh_di4h/notebooks/Petitti_2000_2e_ch02/DecisionAnalysisProject/Project.toml`
  [46ada45e] Agents v6.2.10


In [8]:
using Agents
using Random
using Statistics

# Probabilities
const p_E1 = 0.20          
const p_M1_E1_R1 = 0.05    
const p_M1_E1_R0 = 0.33    
const p_D1_M1 = 0.0023     

# Utilities
const U_Alive = 1.0
const U_Dead = 0.0

0.0

In [9]:
@agent struct Patient(NoSpaceAgent)
    revaccinate::Bool 
    exposed::Bool     
    measles::Bool     
    dead::Bool        
    utility::Float64  
end

In [ ]:
function agent_step!(agent::Patient, model)
    # 1. Chance Node E: Exposure
    # Use abmrng(model) to access the random number generator
    rng = abmrng(model)
    
    if rand(rng) < p_E1
        agent.exposed = true
        
        # 2. Chance Node M: Measles
        prob_measles = agent.revaccinate ? p_M1_E1_R1 : p_M1_E1_R0
        
        if rand(rng) < prob_measles
            agent.measles = true
            
            # 3. Chance Node D: Death
            if rand(rng) < p_D1_M1
                agent.dead = true
                agent.utility = U_Dead
            else
                agent.dead = false
                agent.utility = U_Alive
            end
        else
            agent.measles = false
            agent.utility = U_Alive
        end
    else
        agent.exposed = false
        agent.utility = U_Alive
    end
end

agent_step! (generic function with 1 method)

In [11]:
function initialize_model(; n_agents = 100_000, decision_revaccinate = true)
    space = nothing
    # Create model with v6 syntax
    model = StandardABM(Patient, space; rng = MersenneTwister(42), agent_step! = agent_step!)
    
    # Create agents
    for i in 1:n_agents
        add_agent!(model, decision_revaccinate, false, false, false, 0.0)
    end
    
    return model
end

initialize_model (generic function with 1 method)

In [12]:
# --- Scenario 1: R1 (Revaccinate) ---
model_R1 = initialize_model(n_agents = 1_000_000, decision_revaccinate = true)
run!(model_R1, 1) 

eu_R1 = mean(model_R1[id].utility for id in allids(model_R1))

# --- Scenario 2: R0 (Do Not Revaccinate) ---
model_R0 = initialize_model(n_agents = 1_000_000, decision_revaccinate = false)
run!(model_R0, 1) 

eu_R0 = mean(model_R0[id].utility for id in allids(model_R0))

# --- Results ---
println("--- Agents.jl Monte Carlo Results ---")
println("R1 (Revaccinate) Expected Utility: $eu_R1")
println("R0 (No Revax)    Expected Utility: $eu_R0")

--- Agents.jl Monte Carlo Results ---
R1 (Revaccinate) Expected Utility: 0.999968
R0 (No Revax)    Expected Utility: 0.999843
